In [ ]:
%py
# PySpark code to mask the last 4 digits of invoice numbers in d_product_revenue_clone table

from pyspark.sql.functions import when, lit, length, substring, col

try:
    # Load data from d_product_revenue (replace with your actual table path if it's in a file)
    df = spark.table("d_product_revenue")  # Assuming it's a table
except Exception as e:
    print(f"Error loading data: {e}")
    # Handle the error appropriately (e.g., exit, log, use a default DataFrame)
    # Example: create an empty DataFrame with the correct schema if the table doesn't exist
    from pyspark.sql.types import StructType, StructField, StringType, LongType  # Import required types
    schema = StructType([
        StructField("invoice_number", StringType(), True),
        # Add other columns from d_product_revenue schema here
    ])
    df = spark.createDataFrame([], schema=schema)

# Create or replace the clone table (drop if it exists)
spark.sql("DROP TABLE IF EXISTS d_product_revenue_clone")
df.write.saveAsTable("d_product_revenue_clone")

# Load data from the clone table (important to reload after creating the clone)
df_clone = spark.table("d_product_revenue_clone")

# Improved Masking Logic (handles NULLs, empty strings, and short invoice numbers)
df_masked = df_clone.withColumn("masked_invoice", when(
    col("invoice_number").isNull() | (col("invoice_number") == ""),  # Handle NULLs and empty strings
    col("invoice_number")  # Keep them as NULLs or empty strings
).otherwise(
    when(
        length(col("invoice_number")) <= 4,
        lit("****")  # Mask entirely if <= 4 chars
    ).otherwise(
        substring(col("invoice_number"), 1, length(col("invoice_number")) - 4) + lit("****")  # Mask last 4
    )
))



# Overwrite the invoice_number column in d_product_revenue_clone with the masked values
df_masked = df_masked.drop("invoice_number").withColumnRenamed("masked_invoice","invoice_number") # Drop original and rename

df_masked.write.mode("overwrite").saveAsTable("d_product_revenue_clone")


# Perform Validations against the modified table
# ... (validation code from the provided test cases can be added here)


# Example data type validation after overwriting the column:
result_schema = spark.table("d_product_revenue_clone").schema
invoice_data_type = result_schema["invoice_number"].dataType
assert str(invoice_data_type) == "StringType", f"Data type validation failed. Expected StringType, but got {invoice_data_type}"

# Add more validation tests as needed

#Final comment indicating successful execution

